In [390]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import random
# Global variables
N = 20
M = 8 # number of message bits
q = 97
debug = False
offset = 0
s_A = [np.random.randint(0, 1000) for _ in range(M)]
s_error = [np.random.randint(0, 1000) for _ in range(M)]
s_s = [np.random.randint(0, 1000) for _ in range(M)]
s_s1 = [np.random.randint(0, 1000) for _ in range(M)]
s_e1 = [np.random.randint(0, 1000) for _ in range(M)]
s_e2 = [np.random.randint(0, 1000) for _ in range(M)]
s_m = np.random.randint(0, 1000)
"""
s+A = [10, 15, 20, 25, 30]
s_error = [20, 21, 22, 23, 24]
s_s = [30, 31, 32, 33, 34]
s_s1 = [40, 41, 42, 43, 44]
s_e1 = [50, 51, 52, 53, 54]
s_e2 = [110, 111, 112, 113, 114]
"""


'\ns+A = [10, 15, 20, 25, 30]\ns_error = [20, 21, 22, 23, 24]\ns_s = [30, 31, 32, 33, 34]\ns_s1 = [40, 41, 42, 43, 44]\ns_e1 = [50, 51, 52, 53, 54]\ns_e2 = [110, 111, 112, 113, 114]\n'

# Learning With Errors
$A*s + e = b \in Z_{q}^{M}$<br>
$Z_q=[-\frac{q-1}{2}, \frac{q-1}{2}]$<br>
Here assume errors $e\leftarrow \chi_{e}^{N}$ , which means$\{-1, 0, 1\}^{N}$<br>
Using function: np.random.binomial(n=2, p=0.5, size=M)-1<br>
Same as above, for secret $s\leftarrow \chi_{s}^{N}$<br>
np.random.binomial(n=q-1, p=0.5, size=N)$\ -\ \frac{q-1}{2}$<br>
For public key A array, $A \in Z_{q}^{N\times N}$<br>
A=np.array(\[np.random.randint(0, q) for i in range(N)\] for j in range(M))\ ($A\ is\ N\times N$)

# Linder-Peikert public key encryption scheme
secret key here is choosen from binomial distribution

In [349]:
def KeyGen(seed_A=10, seed_error=20, seed_s=30):
    np.random.seed(seed_A)
    A = np.array([[np.random.randint(0, q) for _ in range(N)] for __ in range(N)])
    np.random.seed(seed_error)
    e = np.random.binomial(n=2, p=0.5, size=N)-1
    if debug:
        print("error = \n", e)
    np.random.seed(seed_s)
    s = np.random.binomial(n=2, p=0.5, size=N)-1
    if np.all(s==0):
        np.random.seed(seed_s + seed_A)
        s = np.random.binomial(n=2, p=0.5, size=N)-1
    # Public key b ---->
    b = (np.matmul(A, s) + e) % q

    return (A, b), s

In [350]:
def encode(x): # x in {0, 1}
    return (q//2)*x

def decode(c_v): # c_v is c-v
    """
    if -q/4 < c_v and c_v < q/4:
        return 0
    elif c_v > q/4 or c_v < -q/4:
        return 1
    else:
        print("error!!!!!!!!")
        return -9999
    """
    return abs(int(round(c_v/(q//2))))

def Enc(pk, m, seed_s1=77, seed_e1=99, seed_e2=111): # public key = (A,b)  message m in {0, 1}
    np.random.seed(seed_s1)
    s1 = np.random.binomial(n=2, p=0.5, size=N)-1
    np.random.seed(seed_e1)
    e1 = np.random.binomial(n=2, p=0.5, size=N)-1
    np.random.seed(seed_e2)
    e2 = np.random.binomial(n=2, p=0.5)-1
    
    A = pk[0]
    b = pk[1]
    b1 = (np.matmul(s1, A) + e1) % q
    
    # inner product <r, b>
    v1 = (np.inner(s1, b) + e2) % q
    c = (encode(m) + v1) % q
    
    if debug:
        print(s1, e1, e2)
        print("b1 = \n", b1)
        print("v1 = \n", v1)
    
    return (b1, c)
def Dec(s, cipher): # cipher = (b1, c)
    b1 = cipher[0]
    c  = cipher[1]
    v = np.inner(b1, s) % q
    if debug:
        print(v)
    
    return decode(c-v)

# Key Generation

In [391]:
# Key Generation part
PK = []
SK = []
for i in range(M):
    pk, sk = KeyGen(s_A[i]+offset, s_error[i]+offset, s_s[i]+offset)
    #pk, sk = KeyGen()
    PK.append(pk)
    SK.append(sk)
    if debug:
        print(pk)
        print(sk)


In [392]:
np.random.seed(seed_m)
m = np.array([np.random.choice(2) for _ in range(M)])
print("message m is:\n", m)

Cipher = []
for i in range(len(m)):
    print("bit %d:" % (i))
    cipher_text = Enc(PK[i], m[i], s_s1[i]+offset, s_e1[i]+offset, s_e2[i]+offset)
    #cipher_text = Enc(PK[i], m[i])
    Cipher.append(cipher_text)
    print("cipher text c is:\n", cipher_text)

message m is:
 [0 1 1 0 0 1 1 1]
bit 0:
cipher text c is:
 (array([37, 10, 78, 13, 71, 76, 48, 55,  1, 37, 64, 56, 66, 25, 74, 36, 47,
       54, 24, 44]), 27)
bit 1:
cipher text c is:
 (array([32, 25, 75, 86, 38, 41, 56, 17, 84, 44, 43, 25, 79, 84, 90, 14, 80,
        7,  6, 55]), 45)
bit 2:
cipher text c is:
 (array([85, 85, 78, 14,  4, 21, 50, 56, 14, 60, 94, 56, 60, 53, 28, 60, 65,
       34, 65, 96]), 86)
bit 3:
cipher text c is:
 (array([40,  5, 24, 26,  6, 18, 48, 78, 93, 13, 87, 89, 36, 51, 12, 69, 62,
       89, 44,  0]), 47)
bit 4:
cipher text c is:
 (array([61, 58, 66, 44, 96, 25, 56, 57, 83, 12, 31, 75, 21, 45, 49, 78, 85,
       80, 62, 17]), 85)
bit 5:
cipher text c is:
 (array([78, 50, 96, 95, 14, 17, 79, 45, 23, 53, 95,  7, 68,  8, 81, 45, 57,
       87, 51, 86]), 38)
bit 6:
cipher text c is:
 (array([43, 61, 65, 27, 53, 73, 50, 11, 61, 39, 66, 44, 41, 85, 74, 14, 84,
       70,  0, 42]), 84)
bit 7:
cipher text c is:
 (array([80, 57, 50, 76, 22, 95, 42, 92, 20, 88, 55, 

In [393]:
m_ = []
for i in range(len(Cipher)):
    m_.append(Dec(SK[i], Cipher[i]))
print("Decode result is: ", m_)
print("Decode success? ", m_ == m)

Decode result is:  [0, 1, 1, 0, 0, 1, 1, 1]
Decode success?  [ True  True  True  True  True  True  True  True]
